<a href="https://colab.research.google.com/github/wzz010624/Scigraphexp/blob/main/subgraph_data_create_new_forml%2Bdl_version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=c6ef1f8c331cde30e41f22af648b4eb50d4094293915208578d8aee5f991ade6
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [ ]:
import pickle
#coauthor_dict[author_id][coauthor_id][paper_year] = [paper_id]
# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/merged_network_with_feature', 'rb') as f:
    merged_network = pickle.load(f)

In [ ]:
import pickle
#coauthor_dict[author_id][coauthor_id][paper_year] = [paper_id]
# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/target_authors_by_year', 'rb') as f:
    target_authors_by_year = pickle.load(f)

In [ ]:
import pickle
import numpy as np
with open('/content/drive/MyDrive/GNNdataset/author_h_index_sequence.pickle', 'rb') as f:
    author_h_index_sequence = pickle.load(f)
# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/authors_high_impact_sequence.pickle', 'rb') as f:
    authors_high_impact_sequence = pickle.load(f)

# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/author_publication_sequence.pickle', 'rb') as f:
    author_publication_sequence = pickle.load(f)

# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/author_citation_sequence.pickle', 'rb') as f:
    author_citation_sequence = pickle.load(f)

In [ ]:
# 计算所有目标节点和目标引文
all_target_nodes = []
all_target_publication = []
all_target_citations = []
all_target_high_impact = []
all_target_h_index = []
all_target_variables = []
for year, target_authors in target_authors_by_year.items():
    for author_id in target_authors:
        publication_years = sorted(author_publication_sequence[author_id].keys())
        career_start_year = publication_years[0]
        career_end_year = publication_years[-1]
        target_year = career_start_year + 14
        accumulated_publication = sum([author_publication_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
        accumulated_citations = sum([author_citation_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
        accumulated_high_impact = sum([authors_high_impact_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
        all_target_nodes.append(f"{author_id}_{year}")
        all_target_citations.append(accumulated_citations)
        all_target_publication.append(accumulated_publication)
        all_target_h_index.append(author_h_index_sequence[author_id].get(target_year, 0))
        all_target_variables.append([accumulated_publication, accumulated_citations, accumulated_high_impact, author_h_index_sequence[author_id].get(target_year, 0)])

In [ ]:
import torch
def edge_list_to_index_tensor(edge_list, node_id_to_int):
    index_list = [(node_id_to_int[src], node_id_to_int[dst]) for src, dst in edge_list]
    index_tensor = torch.tensor(index_list, dtype=torch.long).t().contiguous()
    return index_tensor
def extract_edge_features(edge_list, network):
    edge_features = []
    for src, dst in edge_list:
        edge_data = network.edges[src, dst]
        edge_features.append([edge_data['total_collaboration_count'], edge_data['collaboration_stability']])
    return edge_features

In [ ]:
from torch_geometric.data import Data

def create_train_test_masks(target_nodes, target_citations, train_ratio=0.8):
    num_nodes = len(target_nodes)
    num_train = int(num_nodes * train_ratio)
    
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    
    indices = torch.randperm(num_nodes)
    train_indices = indices[:num_train]
    test_indices = indices[num_train:]
    
    train_mask[train_indices] = True
    test_mask[test_indices] = True
    
    return train_mask, test_mask

In [ ]:
import torch
import numpy as np

def get_neighbors(merged_network, node_id):
    neighbors = list(merged_network.neighbors(node_id))
    return neighbors

def compute_features_average(merged_network, neighbors):
    features = [np.array(list(merged_network.nodes[neighbor].values())) for neighbor in neighbors]
    return np.mean(features, axis=0)

def compute_edge_features_average(merged_network, node_id, neighbors):
    edge_features = [np.array(list(merged_network.edges[node_id, neighbor].values())) for neighbor in neighbors]
    return np.mean(edge_features, axis=0)

In [ ]:
from torch_geometric.data import Data
from typing import List

def extract_subgraph(merged_network, target_node: str):
    # Get the neighbors of the target node
    neighbors = get_neighbors(merged_network, target_node)
    
    # Include the target node in the list
    nodes = [target_node] + neighbors
    
    # Create subgraph
    subgraph = merged_network.subgraph(nodes)
    
    return subgraph
    
def subgraph_to_pyg_data(subgraph, target_node, target_variables):
    # The same code as before, until the definition of y.
    node_features = []
    subgraph_node_id_to_int = {}
    
    # Ensure that the target_node is always the first node in the subgraph.nodes list
    nodes = [target_node] + [node for node in subgraph.nodes if node != target_node]
    
    for i, merged_node_id in enumerate(nodes):
        node_features.append(list(subgraph.nodes[merged_node_id].values()))
        subgraph_node_id_to_int[merged_node_id] = i

    node_features = torch.tensor(node_features, dtype=torch.float)
    edge_index = edge_list_to_index_tensor(list(subgraph.edges), subgraph_node_id_to_int)
    edge_features = extract_edge_features(list(subgraph.edges), subgraph)
    edge_attr = torch.tensor(edge_features, dtype=torch.float)
    y = torch.tensor(target_variables, dtype=torch.float)

    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=y)
    return data

In [ ]:
from torch_geometric.data import Data
from typing import List
import pandas as pd

def create_subgraph_pyg_data_list_and_df(merged_network, target_nodes: List[str], target_variables: List[List[float]]):
    subgraph_pyg_data_list = []
    machine_learning_data = []

    merged_node_id_to_int = {merged_node_id: i for i, merged_node_id in enumerate(merged_network.nodes)}

    # The same code as before, until the for loop.
    for i, target_node in enumerate(target_nodes):
        subgraph = extract_subgraph(merged_network, target_node)
        # Pass all target variables to the subgraph_to_pyg_data function.
        subgraph_pyg_data = subgraph_to_pyg_data(subgraph, target_node, target_variables[i])
        subgraph_pyg_data_list.append(subgraph_pyg_data)

        # Compute average features for neighbors
        neighbors = get_neighbors(merged_network, target_node)
        neighbor_features_avg = compute_features_average(merged_network, neighbors)
        edge_features_avg = compute_edge_features_average(merged_network, target_node, neighbors)

        # Get target node features
        target_node_features = subgraph_pyg_data.x[0].numpy()

        # Add all target variables to the row.
        row = [target_node, *target_node_features, *neighbor_features_avg, *edge_features_avg, target_variables[i]]
        machine_learning_data.append(row)
    # The rest of the function stays the same.
    # Create DataFrame
    columns = ["merged_node_id"] + [f"node_feature_{i}" for i in range(target_node_features.shape[0])] + \
              [f"neighbor_feature_{i}" for i in range(target_node_features.shape[0])] + \
              [f"edge_feature_{i}" for i in range(subgraph_pyg_data.edge_attr.shape[1])] + ["y"]
    machine_learning_df = pd.DataFrame(machine_learning_data, columns=columns)

    return subgraph_pyg_data_list, machine_learning_df

  
subgraph_pyg_data_list, machine_learning_df = create_subgraph_pyg_data_list_and_df(merged_network, all_target_nodes, all_target_variables)

In [ ]:
'''# Calculate the median of y values in the entire dataset
y_values = machine_learning_df['y']
y_median = y_values.median()

# Binarize the y values based on the median
def binarize_y_values(y_values, y_median):
    return (y_values > y_median).astype(int)

# Binarize y values in subgraph_pyg_data_list
for subgraph_data in subgraph_pyg_data_list:
    subgraph_data.y = torch.tensor(binarize_y_values(subgraph_data.y.numpy(), y_median), dtype=torch.float)''

In [ ]:
import pickle

def train_test_split_subgraph_pyg_data_list_and_df(subgraph_pyg_data_list, machine_learning_df, train_ratio=0.8):
    num_nodes = len(subgraph_pyg_data_list)
    num_train = int(num_nodes * train_ratio)
    
    indices = torch.randperm(num_nodes)
    train_indices = indices[:num_train]
    test_indices = indices[num_train:]
    
    # Split subgraph_pyg_data_list
    train_subgraph_pyg_data_list = [subgraph_pyg_data_list[i] for i in train_indices]
    test_subgraph_pyg_data_list = [subgraph_pyg_data_list[i] for i in test_indices]
    
    # Split machine_learning_df
    train_machine_learning_df = machine_learning_df.iloc[train_indices]
    test_machine_learning_df = machine_learning_df.iloc[test_indices]
    
    return train_subgraph_pyg_data_list, test_subgraph_pyg_data_list, train_machine_learning_df, test_machine_learning_df

# Perform train-test split
train_subgraph_pyg_data_list, test_subgraph_pyg_data_list, train_machine_learning_df, test_machine_learning_df = train_test_split_subgraph_pyg_data_list_and_df(subgraph_pyg_data_list, machine_learning_df)

In [ ]:
train_subgraph_pyg_data_list[0].y

tensor([14., 44.,  0.,  4.])

In [ ]:
53f4d1d1dabfaeedd477f8d2

In [ ]:
author_id = "54592ab3dabfaeb0fe334a7e"
publication_years = sorted(author_publication_sequence[author_id].keys())
career_start_year = publication_years[0]
career_end_year = publication_years[-1]
target_year = career_start_year + 14
accumulated_publication = sum([author_publication_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
accumulated_citations = sum([author_citation_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
accumulated_high_impact = sum([authors_high_impact_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
accumulated_publication, accumulated_citations, accumulated_high_impact, author_h_index_sequence[author_id].get(target_year, 0)

(14, 44, 0, 4)

In [ ]:
train_machine_learning_df

,merged_node_id,node_feature_0,node_feature_1,node_feature_2,node_feature_3,node_feature_4,node_feature_5,node_feature_6,node_feature_7,node_feature_8,...,neighbor_feature_18,neighbor_feature_19,neighbor_feature_20,neighbor_feature_21,neighbor_feature_22,neighbor_feature_23,neighbor_feature_24,edge_feature_0,edge_feature_1,y
19813,54592ab3dabfaeb0fe334a7e_1999,1.0,0.2,1.0,0.200000,1.0,0.0,0.0,1.0,0.2,...,12.000000,0.117647,0.250000,0.000000,0.000000,0.000000,0.218182,1.000000,0.000000,"[14, 44, 0, 4]"
71978,53f434eddabfaee2a1cd9156_2009,11.0,2.2,57.0,11.400000,5.0,1.0,0.2,11.0,2.2,...,8.454545,0.326364,0.518477,0.655484,0.422169,0.316796,0.457183,2.090909,0.318182,"[19, 277, 1, 8]"
87123,53f438d5dabfaec09f194596_2010,5.0,1.0,99.0,19.799999,4.0,0.0,0.0,6.0,1.2,...,11.333333,0.386392,0.648679,0.692649,0.495843,0.104625,0.620381,1.833333,0.250000,"[31, 517, 0, 7]"
86162,540923eadabfae8faa66edd6_2010,6.0,1.2,0.0,0.000000,1.0,0.0,0.0,13.0,2.6,...,5.538462,0.164233,0.466283,0.122559,0.170969,0.034188,0.385808,1.230769,0.076923,"[10, 3, 0, 1]"
14063,53f43446dabfaedce551a9b3_1997,8.0,1.6,3.0,0.600000,1.0,0.0,0.0,8.0,1.6,...,9.250000,0.290529,0.455631,0.595907,0.288793,0.041667,0.370614,1.125000,0.125000,"[26, 118, 0, 7]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,53f4476cdabfaee1c0af0890_1982,4.0,0.8,4.0,0.800000,1.0,0.0,0.0,3.0,0.6,...,6.666667,0.252057,0.315593,0.254960,0.000000,0.000000,0.242593,1.000000,0.000000,"[18, 122, 2, 5]"
54072,53f39787dabfae4b34a7c1c2_2006,4.0,0.8,4.0,0.800000,1.0,0.0,0.0,3.0,0.6,...,7.666667,0.366182,0.640241,0.624763,0.354814,0.083333,0.562805,1.666667,0.333333,"[70, 509, 2, 13]"
58851,53f58e47dabfaed7b3f8045b_2007,12.0,2.4,61.0,12.200000,4.0,0.0,0.0,11.0,2.2,...,5.000000,0.410241,0.633509,0.492344,0.303659,0.142123,0.504877,2.454545,0.227273,"[56, 685, 2, 14]"
43544,53f44a5edabfaeb1a7c3ca29_2004,2.0,0.4,4.0,0.800000,1.0,0.0,0.0,2.0,0.4,...,15.000000,0.356750,0.417199,0.388943,0.269347,0.000000,0.431461,1.000000,0.000000,"[16, 59, 1, 5]"


In [ ]:
y_values = machine_learning_df['y']
y_quantile_95 = y_values.quantile(0.95)
def binarize_y_values(y_values, y_quantile_95):
    return (y_values > y_quantile_95).astype(int)
for subgraph_data in subgraph_pyg_data_list:
    subgraph_data.y = torch.tensor(binarize_y_values(subgraph_data.y.numpy(), y_quantile_95), dtype=torch.float)

In [ ]:
train_machine_learning_df['y'] = binarize_y_values(train_machine_learning_df['y'], y_quantile_95)
test_machine_learning_df['y'] = binarize_y_values(test_machine_learning_df['y'], y_quantile_95)

<ipython-input-19-5b57f66d9484>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_machine_learning_df['y'] = binarize_y_values(train_machine_learning_df['y'], y_quantile_95)
<ipython-input-19-5b57f66d9484>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_machine_learning_df['y'] = binarize_y_values(test_machine_learning_df['y'], y_quantile_95)


In [ ]:
# Function to print 0 and 1 distribution
def print_distribution(dataset_name, y_values):
    distribution = pd.Series(y_values).value_counts(normalize=True)  
    # Print the distribution
    print(f"{dataset_name} distribution:")
    print(distribution)
train_y_values = torch.stack([data.y[0] for data in train_subgraph_pyg_data_list])
test_y_values = torch.stack([data.y[0] for data in test_subgraph_pyg_data_list])
print_distribution("Train subgraph_pyg_data_list", pd.Series(train_y_values.numpy()))
print_distribution("Test subgraph_pyg_data_list", pd.Series(test_y_values.numpy()))

Train subgraph_pyg_data_list distribution:
10.0     0.061351
11.0     0.057060
12.0     0.052390
13.0     0.047950
14.0     0.044077
           ...   
275.0    0.000013
285.0    0.000013
365.0    0.000013
511.0    0.000013
283.0    0.000013
Length: 292, dtype: float64
Test subgraph_pyg_data_list distribution:
10.0     0.060837
11.0     0.058677
12.0     0.051660
13.0     0.044696
14.0     0.043347
           ...   
267.0    0.000054
464.0    0.000054
234.0    0.000054
245.0    0.000054
208.0    0.000054
Length: 221, dtype: float64


In [ ]:
train_subgraph_pyg_data_list[0].y[0]

tensor(14.)

In [ ]:
def print_distribution(dataset_name, y_values):
    # Get the first dimension of y_values
    first_dimension_values = [y[0] for y in y_values]
    
    # Compute the distribution
    distribution = pd.Series(first_dimension_values).value_counts(normalize=True)
    
    # Print the distribution
    print(f"{dataset_name} distribution:")
    print(distribution)
print_distribution("Train machine_learning_df", train_machine_learning_df['y'])
print_distribution("Test machine_learning_df", test_machine_learning_df['y'])

Train machine_learning_df distribution:
10     0.061351
11     0.057060
12     0.052390
13     0.047950
14     0.044077
         ...   
275    0.000013
285    0.000013
365    0.000013
511    0.000013
283    0.000013
Length: 292, dtype: float64
Test machine_learning_df distribution:
10     0.060837
11     0.058677
12     0.051660
13     0.044696
14     0.043347
         ...   
267    0.000054
464    0.000054
234    0.000054
245    0.000054
208    0.000054
Length: 221, dtype: float64


In [ ]:
# Save the data to a file
with open('/content/drive/MyDrive/raw_train_test_split_data_full4dimension.pkl', 'wb') as f:
    pickle.dump({
        'train_subgraph_pyg_data_list': train_subgraph_pyg_data_list,
        'test_subgraph_pyg_data_list': test_subgraph_pyg_data_list,
        'train_machine_learning_df': train_machine_learning_df,
        'test_machine_learning_df': test_machine_learning_df,
    }, f)